In [46]:
import numpy as np
import cv2

In [47]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [48]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------

    invK = np.linalg.inv(K)
    depth_masked = depth[seg == 255]
    idx = np.nonzero(seg == 255)

    uv = np.stack([idx[1], idx[0], np.ones((len(idx[0])))], axis=0)

    pc = np.matmul(invK, uv).T * depth_masked.reshape(-1, 1)

    return pc


partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [56]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------

N = partial_pc_sampled.shape[0]
partial_pc_sampled = np.repeat(partial_pc_sampled.reshape(-1, 1, 3), N, 1)
full_pc_sampled = np.repeat(full_pc_sampled.reshape(1, -1, 3), N, 0)
dist = np.linalg.norm(partial_pc_sampled - full_pc_sampled, axis=2)
dist_min = np.min(dist, axis=1)
one_way_CD = np.mean(dist_min).reshape(1)

print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

one way chamfer distance:  [0.00977891]
